In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '../..'))
	print(os.getcwd())
except:
	pass


 # 1. 事前に準備済みのK8s環境を確認

 ## K8s

In [29]:
%%bash
az aks show -g rg_bdc -n o9o9aks


Name     Location    ResourceGroup    KubernetesVersion    ProvisioningState    Fqdn
-------  ----------  ---------------  -------------------  -------------------  -----------------------------------------------------
o9o9aks  japaneast   rg_bdc           1.14.6               Succeeded            o9o9aks-rgbdc-745830-7ebfd297.hcp.japaneast.azmk8s.io


In [30]:
%%bash
az aks browse -g rg_bdc --listen-address 0.0.0.0 -n o9o9aks


Process is interrupted.


 ## Node

In [31]:
%%bash
kubectl get node -o wide


NAME                       STATUS   ROLES   AGE   VERSION   INTERNAL-IP   EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION      CONTAINER-RUNTIME
aks-nodepool1-15232104-0   Ready    agent   63d   v1.14.6   10.240.0.4    <none>        Ubuntu 16.04.6 LTS   4.15.0-1059-azure   docker://3.0.6
aks-nodepool1-15232104-1   Ready    agent   63d   v1.14.6   10.240.0.6    <none>        Ubuntu 16.04.6 LTS   4.15.0-1059-azure   docker://3.0.6
aks-nodepool1-15232104-2   Ready    agent   63d   v1.14.6   10.240.0.5    <none>        Ubuntu 16.04.6 LTS   4.15.0-1059-azure   docker://3.0.6


In [32]:
%%bash
az vm list -g MC_RG_BDC_O9O9AKS_JAPANEAST -d


Name                      ResourceGroup                PowerState    PublicIps    Fqdns    Location    Zones
------------------------  ---------------------------  ------------  -----------  -------  ----------  -------
aks-nodepool1-15232104-0  MC_RG_BDC_O9O9AKS_JAPANEAST  VM running                          japaneast
aks-nodepool1-15232104-1  MC_RG_BDC_O9O9AKS_JAPANEAST  VM running                          japaneast
aks-nodepool1-15232104-2  MC_RG_BDC_O9O9AKS_JAPANEAST  VM running                          japaneast


In [33]:
%%bash
kubectl describe node


Name:               aks-nodepool1-15232104-0
Roles:              agent
Labels:             agentpool=nodepool1
                    beta.kubernetes.io/arch=amd64
                    beta.kubernetes.io/instance-type=Standard_B16ms
                    beta.kubernetes.io/os=linux
                    failure-domain.beta.kubernetes.io/region=japaneast
                    failure-domain.beta.kubernetes.io/zone=1
                    kubernetes.azure.com/cluster=MC_rg_bdc_o9o9aks_japaneast
                    kubernetes.azure.com/role=agent
                    kubernetes.io/arch=amd64
                    kubernetes.io/hostname=aks-nodepool1-15232104-0
                    kubernetes.io/os=linux
                    kubernetes.io/role=agent
                    node-role.kubernetes.io/agent=
                    storageprofile=managed
                    storagetier=Premium_LRS
Annotations:        node.alpha.kubernetes.io/ttl: 0
                    volumes.kubernetes.io/controller-managed-attach-det

 # 2. PV確認

In [34]:
%%bash
cat ~/Jupyter/SQL_Server_on_K8s/FCI/pvc.yaml


kind: StorageClass
apiVersion: storage.k8s.io/v1beta1
metadata:
  name: azure-disk
provisioner: kubernetes.io/azure-disk
parameters:
  storageaccounttype: Standard_LRS
  kind: Managed
---
kind: PersistentVolumeClaim
apiVersion: v1
metadata:
  name: mssql-data
  annotations:
    volume.beta.kubernetes.io/storage-class: azure-disk
spec:
  accessModes:
  - ReadWriteOnce
  resources:
    requests:
      storage: 8Gi

In [35]:
%%bash
kubectl get sc -o wide
kubectl describe sc azure-disk


NAME                PROVISIONER                AGE
azure-disk          kubernetes.io/azure-disk   22h
default (default)   kubernetes.io/azure-disk   30h
managed-premium     kubernetes.io/azure-disk   30h
Name:            azure-disk
IsDefaultClass:  No
Annotations:     kubectl.kubernetes.io/last-applied-configuration={"apiVersion":"storage.k8s.io/v1beta1","kind":"StorageClass","metadata":{"annotations":{},"name":"azure-disk"},"parameters":{"kind":"Managed","storageaccounttype":"Standard_LRS"},"provisioner":"kubernetes.io/azure-disk"}

Provisioner:           kubernetes.io/azure-disk
Parameters:            kind=Managed,storageaccounttype=Standard_LRS
AllowVolumeExpansion:  <unset>
MountOptions:          <none>
ReclaimPolicy:         Delete
VolumeBindingMode:     Immediate
Events:                <none>


In [36]:
%%bash
kubectl get pv -o wide -n mssql-fci
kubectl describe pv -n mssql-fci


fd1c1c747/resourceGroups/mc_rg_bdc_o9o9aks_japaneast/providers/Microsoft.Compute/disks/kubernetes-dynamic-pvc-94010172-0d89-11ea-8d22-169104339902
    Kind:         Managed
    FSType:       
    CachingMode:  ReadOnly
    ReadOnly:     false
Events:           <none>


Name:            pvc-995ac777-0d89-11ea-8d22-169104339902
Labels:          <none>
Annotations:     pv.kubernetes.io/bound-by-controller: yes
                 pv.kubernetes.io/provisioned-by: kubernetes.io/azure-disk
                 volumehelper.VolumeDynamicallyCreatedByKey: azure-disk-dynamic-provisioner
Finalizers:      [kubernetes.io/pv-protection]
StorageClass:    default
Status:          Bound
Claim:           mssql-cluster/data-storage-0-0
Reclaim Policy:  Delete
Access Modes:    RWO
VolumeMode:      Filesystem
Capacity:        15Gi
Node Affinity:   <none>
Message:         
Source:
    Type:         AzureDisk (an Azure Data Disk mount on the host and bind mount to the pod)
    DiskName:     kubernetes-dynamic-pvc-

In [37]:
%%bash
kubectl get pvc -o wide -n mssql-fci
kubectl describe pvc -n mssql-fci


NAME         STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   AGE
mssql-data   Bound    pvc-df8c63bc-0cd0-11ea-9665-068915c15dce   8Gi        RWO            azure-disk     22h
Name:          mssql-data
Namespace:     mssql-fci
StorageClass:  azure-disk
Status:        Bound
Volume:        pvc-df8c63bc-0cd0-11ea-9665-068915c15dce
Labels:        <none>
Annotations:   kubectl.kubernetes.io/last-applied-configuration:
                 {"apiVersion":"v1","kind":"PersistentVolumeClaim","metadata":{"annotations":{"volume.beta.kubernetes.io/storage-class":"azure-disk"},"name...
               pv.kubernetes.io/bind-completed: yes
               pv.kubernetes.io/bound-by-controller: yes
               volume.beta.kubernetes.io/storage-class: azure-disk
               volume.beta.kubernetes.io/storage-provisioner: kubernetes.io/azure-disk
Finalizers:    [kubernetes.io/pvc-protection]
Capacity:      8Gi
Access Modes:  RWO
VolumeMode:    Filesystem
Mounted

In [38]:
%%bash
az disk list -g MC_RG_BDC_O9O9AKS_JAPANEAST 


Name                                                                ResourceGroup                Location    Zones    Sku           OsType    SizeGb    ProvisioningState
------------------------------------------------------------------  ---------------------------  ----------  -------  ------------  --------  --------  -------------------
aks-nodepool1-15232104-0_OsDisk_1_233442b330ce48aeb803829051658ad3  MC_RG_BDC_O9O9AKS_JAPANEAST  japaneast            Premium_LRS   Linux     100       Succeeded
aks-nodepool1-15232104-1_OsDisk_1_1f8b8034fcce4ee2b97c024821b6d8bf  MC_RG_BDC_O9O9AKS_JAPANEAST  japaneast            Premium_LRS   Linux     100       Succeeded
aks-nodepool1-15232104-2_OsDisk_1_e0e8ca95bf704caeade77ebf95d47a82  MC_RG_BDC_O9O9AKS_JAPANEAST  japaneast            Premium_LRS   Linux     100       Succeeded
kubernetes-dynamic-pvc-246beaa7-0d88-11ea-8d22-169104339902         MC_RG_BDC_O9O9AKS_JAPANEAST  japaneast            Standard_LRS            10        Succeeded
kubernetes

 # 3. Pod確認

In [39]:
%%bash
cat ~/Jupyter/SQL_Server_on_K8s/FCI/sql19deployment.yaml


apiVersion: apps/v1beta1
kind: Deployment
metadata:
  name: mssql-deployment
spec:
  replicas: 1
  template:
    metadata:
      labels:
        app: mssql
    spec:
      terminationGracePeriodSeconds: 10
      containers:
      - name: mssql
        image: mcr.microsoft.com/mssql/server:2019-CU1
        ports:
        - containerPort: 1433
        env:
        - name: MSSQL_PID
          value: "Developer"
        - name: ACCEPT_EULA
          value: "Y"
        - name: MSSQL_SA_PASSWORD
          valueFrom:
            secretKeyRef:
              name: mssql
              key: SA_PASSWORD 
        volumeMounts:
        - name: mssqldb
          mountPath: /var/opt/mssql
      volumes:
      - name: mssqldb
        persistentVolumeClaim:
          claimName: mssql-data
---
apiVersion: v1
kind: Service
metadata:
  name: mssql-deployment
spec:
  selector:
    app: mssql
  ports:
    - protocol: TCP
      port: 1433
      targetPort: 1433
  type: LoadBalancer

In [40]:
%%bash
kubectl get pod -o wide -n mssql-fci
kubectl describe pod -n mssql-fci


NAME                                READY   STATUS    RESTARTS   AGE   IP            NODE                       NOMINATED NODE   READINESS GATES
mssql-deployment-79fdff6b95-nk8zx   1/1     Running   0          18h   10.244.1.65   aks-nodepool1-15232104-2   <none>           <none>
Name:           mssql-deployment-79fdff6b95-nk8zx
Namespace:      mssql-fci
Priority:       0
Node:           aks-nodepool1-15232104-2/10.240.0.5
Start Time:     Fri, 22 Nov 2019 06:37:16 +0000
Labels:         app=mssql
                pod-template-hash=79fdff6b95
Annotations:    <none>
Status:         Running
IP:             10.244.1.65
IPs:            <none>
Controlled By:  ReplicaSet/mssql-deployment-79fdff6b95
Containers:
  mssql:
    Container ID:   docker://58924c11e259f4b452a80eed0668da32c838ac00d08317257e342e0d7bc2cf66
    Image:          mcr.microsoft.com/mssql/server:2019-RC1
    Image ID:       docker-pullable://mcr.microsoft.com/mssql/server@sha256:a11facbda1b1cc299d4a37499ef79cd18e38bfb8e5dd7e45cc

In [41]:
%%bash
kubectl get service -o wide -n mssql-fci
kubectl describe service -n mssql-fci


NAME               TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)          AGE   SELECTOR
mssql-deployment   LoadBalancer   10.0.109.226   52.185.174.22   1433:30729/TCP   22h   app=mssql
Name:                     mssql-deployment
Namespace:                mssql-fci
Labels:                   <none>
Annotations:              kubectl.kubernetes.io/last-applied-configuration:
                            {"apiVersion":"v1","kind":"Service","metadata":{"annotations":{},"name":"mssql-deployment","namespace":"mssql-fci"},"spec":{"ports":[{"por...
Selector:                 app=mssql
Type:                     LoadBalancer
IP:                       10.0.109.226
LoadBalancer Ingress:     52.185.174.22
Port:                     <unset>  1433/TCP
TargetPort:               1433/TCP
NodePort:                 <unset>  30729/TCP
Endpoints:                10.244.1.65:1433
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


 # 4. SQL接続テスト

In [42]:
%%bash
kubectl exec mssql-deployment-79fdff6b95-nk8zx -c mssql -n mssql-fci -- /opt/mssql-tools/bin/sqlcmd -U sa -P $PASSWORD -Q 'SELECT @@VERSION'



------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Microsoft SQL Server 2019 (RC1) - 15.0.1900.25 (X64) 
	Aug 16 2019 14:20:53 
	Copyright (C) 2019 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 16.04.6 LTS) <X64>                                                                                                                      

(1 rows affected)


In [43]:
%%bash
sqlcmd -S 52.185.174.22 -U sa -P $PASSWORD  -Q 'select @@version'



------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Microsoft SQL Server 2019 (RC1) - 15.0.1900.25 (X64) 
	Aug 16 2019 14:20:53 
	Copyright (C) 2019 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 16.04.6 LTS) <X64>                                                                                                                      

(1 rows affected)


In [45]:
%%bash
sqlcmd -S 52.185.174.22 -U sa -P $PASSWORD  -Q 'create database FCIDB'
sqlcmd -S 52.185.174.22 -U sa -P $PASSWORD  -Q 'use FCIDB; create table tab01(id int, name varchar(max)); insert into tab01 values (1, "SQL Server runs on K8s!")'
sqlcmd -S 52.185.174.22 -U sa -P $PASSWORD  -Q 'use FCIDB; select * from tab01'


Changed database context to 'FCIDB'.

(1 rows affected)
Changed database context to 'FCIDB'.
id          name                                                                                                                                                                                                                                                            
----------- ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
          1 SQL Server runs on K8s!                                                                                                                                                                                                                                         

(1 rows affected)


 # 5. Pod障害擬似

In [47]:
%%bash
kubectl get pod -o wide -n mssql-fci


NAME                                READY   STATUS    RESTARTS   AGE   IP            NODE                       NOMINATED NODE   READINESS GATES
mssql-deployment-79fdff6b95-nk8zx   1/1     Running   0          18h   10.244.1.65   aks-nodepool1-15232104-2   <none>           <none>


In [49]:
%%bash
kubectl delete pod mssql-deployment-79fdff6b95-nk8zx -n mssql-fci


pod "mssql-deployment-79fdff6b95-nk8zx" deleted


In [50]:
%%bash
kubectl get pod -o wide -n mssql-fci


NAME                                READY   STATUS    RESTARTS   AGE   IP            NODE                       NOMINATED NODE   READINESS GATES
mssql-deployment-79fdff6b95-7p9rh   1/1     Running   0          21s   10.244.1.93   aks-nodepool1-15232104-2   <none>           <none>


In [51]:
%%bash
sqlcmd -S 52.185.174.22 -U sa -P $PASSWORD  -Q 'use FCIDB; select * from tab01'


Changed database context to 'FCIDB'.
id          name                                                                                                                                                                                                                                                            
----------- ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
          1 SQL Server runs on K8s!                                                                                                                                                                                                                                         

(1 rows affected)
